In [23]:
# Dependencies
import tweepy
import json
import numpy as np
import pandas as pd

from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [24]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [29]:
# List of target users
target_user = ["@BBC", "@CBS", "@CNN", "@FoxNews", "@NYTimes"]

# List for dictionaries of results
results_list = []

# Loop through each user
for user in target_user:

    # Counter
    counter = 1

    # Variable for max_id
    oldest_tweet = None
    
    # Loop through 5 pages of tweets (total 100 tweets)
    for _ in range(5):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user, max_id = oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            
            # Create a dictionaty of results
            tweet_entity = {
                "Username": user,
                "Date": tweet["created_at"],
                "Compound": compound,
                "Positive": pos,
                "Negative": neu,
                "Neutral": neg,
                "Tweets Ago": counter
            }

            # Append dictionary to list
            results_list.append(tweet_entity)

            # Get Tweet ID, subtract 1, and assign to oldest_tweet
            oldest_tweet = tweet['id'] - 1
            
            #Add to counter 
            counter += 1

In [30]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(results_list)
sentiments_pd.shape

(500, 7)